# How to run a semantic plugins from file
Now that you're familiar with Kernel basics, let's see how the kernel allows you to run Semantic Plugins and Semantic Functions stored on disk. 

A Semantic Plugin is a collection of Semantic Functions, where each function is defined with natural language that can be provided with a text file. 

Refer to our [glossary](../../docs/GLOSSARY.md) for an in-depth guide to the terms.

The repository includes some examples under the [samples](https://github.com/microsoft/semantic-kernel/tree/main/samples) folder.

For instance, [this](../../samples/plugins/FunPlugin/Joke/skprompt.txt) is the **Joke function** part of the **FunPlugin plugin**:

```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE TOPIC BELOW.
JOKE MUST BE:
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY.
BE CREATIVE AND FUNNY. I WANT TO LAUGH.
+++++
{{$input}}
+++++
```

Note the special **`{{$input}}`** token, which is a variable that is automatically passed when invoking the function, commonly referred to as a "function parameter". 

We'll explore later how functions can accept multiple variables, as well as invoke other functions.


In the same folder you'll notice a second [config.json](../../samples/plugins/FunPlugin/Joke/config.json) file. The file is optional, and is used to set some parameters for large language models like Temperature, TopP, Stop Sequences, etc.

```
{
  "schema": 1,
  "description": "Generate a funny joke",
  "execution_settings": [
    {
      "max_tokens": 1000,
      "temperature": 0.9,
      "top_p": 0.0,
      "presence_penalty": 0.0,
      "frequency_penalty": 0.0
    }
  ]
}
```

Given a semantic function defined by these files, this is how to load and use a file based semantic function.

Configure and create the kernel, as usual, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.23.0

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#!import config/Settings.cs
#!import config/BuilderForOllama.cs

var kernel = BuilderForOllama.BuildKernel("deepseek-r1");


Installed Packages Microsoft.SemanticKernel, 1.23.0

Error: (1,31): error CS0117: 'BuilderForOllama'에는 'BuildKernelWithOllama'에 대한 정의가 포함되어 있지 않습니다.

In [15]:
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#!import config/Settings.cs
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using System.Net.Http;

//Create Kernel builder
var builder = Kernel.CreateBuilder();
// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
Console.WriteLine($"Using model: {model}, {apiKey}, {orgId}");
//if (useAzureOpenAI)
    //builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
//else
// Ollama용 HttpClient 설정
var httpClient = new HttpClient()
{
    BaseAddress = new Uri("http://localhost:11434/v1/")
};
builder.AddOpenAIChatCompletion(modelId: model, apiKey, orgId, httpClient: httpClient);

var kernel = builder.Build();


Installed Packages Microsoft.SemanticKernel, 1.23.0

Using model: deepseek-r1:1.5b, ollama, 


Import the plugin and all its functions:

In [16]:
// FunPlugin directory path
var funPluginDirectoryPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "prompt_template_samples", "FunPlugin");

// Load the FunPlugin from the Plugins Directory
var funPluginFunctions = kernel.ImportPluginFromPromptDirectory(funPluginDirectoryPath);

How to use the plugin functions, e.g. generate a joke about "*time travel to dinosaur age*":

In [17]:
// Construct arguments
var arguments = new KernelArguments() { ["input"] = "time travel to dinosaur age" };

// Run the Function called Joke
var result = await kernel.InvokeAsync(funPluginFunctions["Joke"], arguments);

// Return the result to the Notebook
Console.WriteLine(result);

<think>
Okay, so I need to come up with a joke or humorous story about time travel to the dinosaur age. The user mentioned that it should be graded, workplace or family-safe, no性别偏见或偏见。Also, it needs to be creative and funny, and incorporate the style suggestion which includes "time travel to dinosaur age" and "+++++". 

Hmm, first, I need to think about how time travel works in a fun way. Maybe something that's not too technical but still imaginative. Dinosaur ages are really old, so maybe it's more of a sci-fi twist.

I should avoid any explicit or offensive content. The joke needs to be light-hearted and funny. Let me brainstorm some ideas. 

What if the story is about someone who travels back in time to help others? Maybe a scientist who goes back to prehistoric times to fix something. That could be funny because it's relatable and uses a classic time travel concept.

Wait, but I need to incorporate the style suggestion. So maybe instead of just talking about time travel, include e

Great, now that you know how to load a plugin from disk, let's show how you can [create and run a semantic function inline.](./03-semantic-function-inline.ipynb)